# Guide pymongo

# Connect to database

In [2]:
import pymongo
# Establish a connection to MongoDB
try:
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    print("Connected successfully to MongoDB!")
except pymongo.errors.ConnectionFailure as e:
    print("Could not connect to MongoDB: %s" % e)

Connected successfully to MongoDB!


# Database Overview

In [3]:
client.list_database_names()      
database = client["myschool"]                       # Set the database to work on
collection = database["student"]                    # Checks the database name

['admin', 'config', 'kirins', 'local', 'myschool']

In [10]:
# Count the number of documents in the collection
num_documents = collection.count_documents({})

# Get the unique values of the "category" field in the collection
categories = collection.distinct("category")
attributes = list (collection.find().limit(1)[1])     ## WARNING use as reference , NoSQL db can have different attributes by document

# Print a summary of the dataset content
print(f"Number of documents: {num_documents}")
print(f"Number of categories: {len(categories)}")
print(f"Categories: {categories}")
print ('Attributes names : %s' % attributes)

Number of documents: 5
Number of categories: 0
Categories: []
Attributes names : ['_id', 'id', 'name', 'gender', 'major', 'gpa']


In [32]:
documents = list(collection.find())

# Print the documents
for document in documents:
    print(document)

{'_id': ObjectId('64425183615c6d60a6d91d10'), 'name': 'Ms. Smith', 'gender': 'female', 'department': 'English', 'years_of_experience': 10}
{'_id': ObjectId('6445f2a9cdad944b2ded4150'), 'name': 'Ms. Smith', 'gender': 'female', 'department': 'English', 'years_of_experience': 10}


In [7]:
## Dataset content summary
num_documents = collection.count_documents({'_id' : {'$exists' : 1}})
attributes = list (collection.find().limit(1)[1])     ## WARNING use as reference , NoSQL db can have different attributes by document

print ('Number of documents : %d' % num_documents)
print ('Attributes names : %s' % attributes)

Number of documents : 5
Attributes names : ['_id', 'id', 'name', 'gender', 'major', 'gpa']


In [11]:
## We can list one document
list ( collection.find().limit(1) )

[{'_id': ObjectId('6442402f4a3b5c368cda59c9'),
  'id': 1,
  'name': 'Cedric',
  'gender': 'male',
  'major': 'Mechatronics',
  'gpa': 3.1}]

# Insert


Use the appropriate `insert_one()` or `insert_many()` method to insert the document or documents into the collection.

In [31]:
# Select the database and collection to use
database = client["myschool"]
collection = database["teacher"]

# Define the teacher document to insert
teacher = {"name": "Ms. Smith", "gender": "female", "department": "English", "years_of_experience": 10}

# Insert the teacher document into the collection
result = collection.insert_one(teacher)

# Print the ID of the inserted document
print(result.inserted_id)

6445f2a9cdad944b2ded4150


In [33]:
# Define a list of documents to insert
teachers = [
  { 'name': 'John Doe', 'subject': 'Math' },
  { 'name': 'Jane Smith', 'subject': 'Science' },
  { 'name': 'Bob Johnson', 'subject': 'History' }
]

# Insert the documents into the collection
result = collection.insert_many(teachers)

# Print the IDs of the inserted documents
print(result.inserted_ids)

[ObjectId('6445f609cdad944b2ded4151'), ObjectId('6445f609cdad944b2ded4152'), ObjectId('6445f609cdad944b2ded4153')]


# Update

In [36]:
# Update the department field of Bob Johnson's document
filter = { 'name': 'Bob Johnson' }
update = { '$set': { 'department': 'Social Studies' } }
result = collection.update_one(filter, update)

# Print the number of documents updated
print(result.modified_count)

1


In [43]:
# Update the department field of all History teachers' documents
filter = { 'department': 'English' }
update = { '$set': { 'class': '3C' } }
result = collection.update_many(filter, update)

# Print the number of documents updated
print(result.modified_count)

2


# Delete

In [44]:
# Delete the 'years_of_experience' field from all documents
filter = {}
update = { '$unset': { 'class': '' } }
result = collection.update_many(filter, update)

# Print the number of documents updated
print(result.modified_count)

2


# Data visualization and Queries

In [49]:
# Visualization of attribute names
print (list (collection.find({"years_of_experience" : 10, "gender" : 'female'}).limit(1)[1]))

['_id', 'name', 'gender', 'department', 'years_of_experience']


In [52]:
# Visualization of attribute unique content
query = collection.distinct("years_of_experience")
print ("Years_of_experience :" + str(sorted(query)))

query = collection.distinct("department")
print ("Department :" + str(sorted(query)))

Years_of_experience :[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Department :['Art', 'Computer Science', 'English', 'Foreign Language', 'History', 'Math', 'Music', 'Physical Education', 'Science', 'Social Studies']


In [55]:
list(collection.find({"years_of_experience": 10}))

[{'_id': ObjectId('64425183615c6d60a6d91d10'),
  'name': 'Ms. Smith',
  'gender': 'female',
  'department': 'English',
  'years_of_experience': 10},
 {'_id': ObjectId('6445f2a9cdad944b2ded4150'),
  'name': 'Ms. Kyan',
  'gender': 'female',
  'department': 'Math',
  'years_of_experience': 10},
 {'_id': '4',
  'name': 'Alice Lee',
  'gender': 'female',
  'department': 'English',
  'years_of_experience': 10}]

In [63]:
# Filter by fields
filters = {"department": 'Math'}
fields = ['name','gender', 'department','years_of_experience']

list(collection.find(filters, fields).limit(1))

[{'_id': ObjectId('6445f2a9cdad944b2ded4150'),
  'name': 'Ms. Kyan',
  'gender': 'female',
  'department': 'Math',
  'years_of_experience': 10}]

# Count

In [78]:
collection.count_documents({"years_of_experience": 2})

1

In [76]:
# Maximum value
list(collection.find({},{"id": 0}).sort('years_of_experience', pymongo.DESCENDING).limit(1))

[{'_id': ObjectId('64425183615c6d60a6d91d10'),
  'name': 'Ms. Smith',
  'gender': 'female',
  'department': 'English',
  'years_of_experience': 10}]

In [87]:
max(collection.distinct("years_of_experience"))

10

In [84]:
list(collection.find({},{"id": 0}).sort('years_of_experience', pymongo.ASCENDING).limit(1))

[{'_id': '9',
  'name': 'Michael Johnson',
  'gender': 'male',
  'department': 'Computer Science',
  'years_of_experience': 1}]

In [88]:
min(collection.distinct("years_of_experience"))

1

In [92]:
yearmale   = collection.find({"gender" : 'male'}).distinct("years_of_experience")
yearfemale = collection.find({"gender" : 'female'}).distinct("years_of_experience")

print ('Male -  Min age: ' + str(min(yearmale)) + ' and Max age: ' + str(max(yearmale)))
print ('Female -  Min age: ' + str(min(yearfemale)) + ' and Max age: ' + str(max(yearfemale)))

Male -  Min age: 1 and Max age: 7
Female -  Min age: 6 and Max age: 10
